# K-S statistic is calculated from t-map generated using decoder based perturbation interpretation approach mapping ENDOs to brain regions

In [ ]:
# Imports
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy
from scipy import stats
##imports
import pandas as pd
import os
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
import nibabel as nib
from bs4 import BeautifulSoup
from tqdm import tqdm
# utils contain helper functions to parse regions from the Harvard-Oxford cortical and subcortical atlas
from utils import *  
%load_ext autoreload
%matplotlib inline

In [ ]:
# Path to T1 t-maps generated using decoder based perturbation interpretation approach and selecting all t-maps
path_T1 =  ""
T1 = glob(path_T1+"/*")

In [ ]:
# extracting voxel labels for each regions based on HarvardOxford cortical and subcortical atlases
sub_cortical_atlas = nib.load("HarvardOxford-sub-maxprob-thr50-1mm.nii.gz").get_fdata()
cortical_atlas = nib.load("HarvardOxford-cort-maxprob-thr50-1mm.nii.gz").get_fdata()
cort_labels = "Cortical.xml"
subcort_labels = "HarvardOxford-Subcortical.xml"
#getting label names
cort_regions, cort_labels = get_atlas_region_labels("HarvardOxford-Cortical.xml")
subcort_regions, subcort_labels = get_atlas_region_labels("HarvardOxford-Subcortical.xml")
#getting voxel count for each region
regions_subcort, cnts_subcort = np.unique(sub_cortical_atlas, return_counts=True)
regions_cort, cnts_cort = np.unique(cortical_atlas, return_counts=True)
#changing datatype
regions_subcort = regions_subcort.astype(int)
regions_subcort = regions_subcort.astype(str)
regions_cort = regions_cort.astype(int)
regions_cort = regions_cort.astype(str)
#converting to dataframe
cort_df_cnt = pd.DataFrame(zip(regions_cort, cnts_cort), columns=["regions_cort", "total_counts_cort"])
subcort_df_cnt = pd.DataFrame(zip(regions_subcort, cnts_subcort), columns=["regions_sub", "total_counts_subcort"])
cort_df = pd.DataFrame(zip(cort_labels, cort_regions), columns=["regions_cort", "regions_names"])
subcort_df = pd.DataFrame(zip(subcort_regions, subcort_labels), columns=["regions_names", "regions_sub"])
#final dfs
cortical_regions_df = cort_df.merge(cort_df_cnt, on="regions_cort")
subcort_regions_df = subcort_df.merge(subcort_df_cnt, on="regions_sub")
#left and right regions combined to common structure (eg. left hippocampus and right hippocampus -> hippocampus)
subcort_regions_df.iloc[:,0] = subcort_regions_df.iloc[:,0].apply(lambda x: x.replace('Left', ''))
subcort_regions_df.iloc[:,0] = subcort_regions_df.iloc[:,0].apply(lambda x: x.replace('Right', ''))
total_subcort_voxels = subcort_regions_df.groupby("regions_names")["total_counts_subcort"].sum()
labels_subcort = subcort_regions_df.groupby("regions_names")["regions_sub"].unique()
cortical_regions_df.iloc[:,0] = cortical_regions_df.iloc[:,0].apply(lambda x: x.replace('Left', ''))
cortical_regions_df.iloc[:,0] = cortical_regions_df.iloc[:,0].apply(lambda x: x.replace('Right', ''))
total_cort_voxels = cortical_regions_df.groupby("regions_names")["total_counts_cort"].sum()
labels_cort = cortical_regions_df.groupby("regions_names")["regions_cort"].unique()

In [ ]:
total_cort_voxels

In [ ]:
total_subcort_voxels

In [ ]:
def get_ranks_subcortical(dim):
    """
    Ranking voxels in tmap within Harvard Oxford subcortical atlas
    """
    dim_data = nib.load(dim).get_fdata()
    subcortical = pd.DataFrame(zip(list(sub_cortical_atlas.flatten()), list(dim_data.flatten())), columns=["atlas", "values"])
    subcortical = subcortical [subcortical["atlas"]!=0]
    subcortical["ranks"] = subcortical["values"].rank()
    subcortical.sort_values(by="ranks", inplace=True, ascending=False)
    subcortical.reset_index(drop=True, inplace=True)
    return subcortical

In [ ]:
def get_ranks_cortical(dim):
    """
    Ranking voxels in tmap within Harvard Oxford subcortical atlas
    """
    dim_data = nib.load(dim).get_fdata()
    cortical = pd.DataFrame(zip(list(cortical_atlas.flatten()), list(dim_data.flatten())), columns=["atlas", "values"])
    cortical = cortical[cortical["atlas"]!=0]
    cortical["ranks"] = cortical["values"].rank()
    cortical.sort_values(by="ranks", inplace=True, ascending=False)
    cortical.reset_index(drop=True, inplace=True)
    return cortical

In [ ]:
def enrichment_subcort(dataframe, total_labelled_voxels, total_voxels=1492617):  
    """
    Enrichment function for subcortical region to be used with enrichment_function_subcortical
    """
    vals = []
    s= 0
    for i in dataframe["region"].values:
        if i==1:
            s=s + (1/total_labelled_voxels) - 1/total_voxels
            vals.append(s)
        else:
            s= s + 0 - 1/total_voxels
            vals.append(s)
    return vals

In [ ]:
def enrichment_cort(dataframe, total_labelled_voxels, total_voxels=476094 ):    
    """
    Enrichment function for cortical region to be used with enrichment_function_cortical
    """
    vals = []
    s= 0
    for i in dataframe["region"].values:
        if i==1:
            s=s + (1/total_labelled_voxels) - 1/total_voxels
            vals.append(s)
        else:
            s= s + 0 - 1/total_voxels
            vals.append(s)
    return vals

In [ ]:
def enrichment_function_subcortical(function, subcortical):
    """
    main function to use for subcortical regions
    """
    values = []
    regions = []
    total_voxels = 1492617
    for i in tqdm(range(labels_subcort.shape[0])):
        total_labelled_voxels = total_subcort_voxels.iloc[i]
        region_name = total_subcort_voxels.index[i]
        regions.append(region_name)
        label = list(labels_subcort.iloc[i])
        if region_name=="Brain-Stem":
            subcortical["region"] = subcortical["atlas"].apply(lambda x: 1 if (x == int(label[0])) else 0)
        else:
            subcortical["region"] = subcortical["atlas"].apply(lambda x: 1 if (x == int(label[0]))|(x == int(label[1])) else 0)       
        values.append(function(subcortical,total_labelled_voxels, total_voxels))
    ks = [np.array(i).max() for i in values]
    df = pd.DataFrame(zip(ks, values, regions), columns=["ks","values", "regions"]).sort_values(by="ks", ascending=False).reset_index(drop=True)
    return list(df.ks.values), list(df["values"].values), list(df.regions.values)

In [ ]:
def enrichment_function_cortical(function, cortical):
    """
    main function to use for subcortical regions
    """
    values = []
    regions = []
    total_voxels = 476094
    for i in tqdm(range(labels_cort.shape[0])):
        total_labelled_voxels = total_cort_voxels.iloc[i]
        region_name = total_cort_voxels.index[i]
        regions.append(region_name)
        label = list(labels_cort.iloc[i])
        cortical["region"] = cortical["atlas"].apply(lambda x: 1 if (x == int(label[0])) else 0)       
        values.append(function(cortical,total_labelled_voxels, total_voxels))
    ks = [np.array(i).max() for i in values]
    df = pd.DataFrame(zip(ks, values, regions), columns=["ks","values", "regions"]).sort_values(by="ks", ascending=False).reset_index(drop=True)
    return list(df.ks.values), list(df["values"].values), list(df.regions.values)

In [ ]:
def plot_cortical(values, regions, ks_cort,  dim, mod="T1"):
    """
    Creating plots for subcortical regions
    """
    plt.figure(figsize=(18,16))
    plt.plot(values[0])
    plt.plot(values[1])
    plt.plot(values[2])
    plt.plot(values[3])
    plt.plot(values[4])
    plt.plot(values[5])
    plt.plot(values[6])
    plt.plot(values[7],)
    plt.plot(values[8])
    plt.plot(values[9])
    plt.plot(values[10], color="#414451")
    plt.plot(values[11], color="#cfcfcf")
    plt.plot(values[12], color="#cfcfcf")
    plt.plot(values[13], color="#cfcfcf")
    plt.plot(values[14], color="#cfcfcf")
    plt.plot(values[15], color="#cfcfcf")
    plt.plot(values[16], color="#cfcfcf")
    plt.plot(values[17], color="#cfcfcf")
    plt.plot(values[18], color="#cfcfcf")
    plt.plot(values[19], color="#cfcfcf")
    plt.plot(values[20], color="#cfcfcf")
    plt.plot(values[21], color="#cfcfcf")
    plt.plot(values[22], color="#cfcfcf")
    plt.plot(values[23], color="#cfcfcf")
    plt.plot(values[24], color="#cfcfcf")
    plt.plot(values[25], color="#cfcfcf")
    plt.plot(values[26], color="#cfcfcf")
    plt.plot(values[27], color="#cfcfcf")
    plt.plot(values[28], color="#cfcfcf")
    plt.plot(values[29], color="#cfcfcf")
    plt.plot(values[30], color="#cfcfcf")
    plt.plot(values[31], color="#cfcfcf")
    plt.plot(values[32], color="#cfcfcf")
    plt.plot(values[33], color="#cfcfcf")
    plt.plot(values[34], color="#cfcfcf")
    plt.plot(values[35], color="#cfcfcf")
    plt.plot(values[36], color="#cfcfcf")
    plt.plot(values[37], color="#cfcfcf")
    plt.plot(values[38], color="#cfcfcf")
    plt.plot(values[39], color="#cfcfcf")
    plt.plot(values[40], color="#cfcfcf")
    plt.plot(values[41], color="#cfcfcf")
    plt.plot(values[42], color="#cfcfcf")
    plt.plot(values[43], color="#cfcfcf")
    plt.plot(values[44], color="#cfcfcf")
    plt.plot(values[45], color="#cfcfcf")
    plt.plot(values[46], color="#cfcfcf")
    plt.axhline(color="black", linestyle="--")
    plt.axvline(color="black", linestyle="--")
    plt.xlabel("Number of voxels", fontdict={"fontsize":24})
    legend = [f"{i} (K-S: {np.round(j, 3)})" for i, j in zip(regions, ks_cort)]
    legend = [i for i in legend[:11]]
    legend.append("Other cortical regions")
    plt.legend(legend,loc=1)
    plt.savefig(f"cortical_{mod}_{dim}")
    plt.close()

In [ ]:
def plot_subcortical(values, regions, ks_subcort, dim, mod="T1"):
    """
    Creating plots for subcortical regions
    """
    plt.figure(figsize=(18,12))
    plt.plot(values[0], )
    plt.plot(values[1],)
    plt.plot(values[2],  )
    plt.plot(values[3],  )
    plt.plot(values[4],  )
    plt.plot(values[5],  )
    plt.plot(values[6],  )
    plt.plot(values[7],)
    plt.plot(values[8],)
    plt.plot(values[9],  )
    plt.plot(values[10],color="#414451")
    plt.axhline(color="black", linestyle="--")
    plt.axvline(color="black", linestyle="--")
    plt.xlabel("Number of voxels", fontdict={"fontsize":24})
    legend = [f"{i} (K-S: {np.round(j, 3)})" for i, j in zip(regions, ks_subcort)]
    plt.legend(legend,loc=1)
    plt.savefig(f"subcortical_{mod}_{dim}")
    plt.close()

In [ ]:
# Saving plots
save_path = "T1"
os.chdir(save_path)
for i in tqdm(range(len(T1))):
    dim = T1[i].split("/")[-1].split(".")[0].split("_")[-1]
    print("i", i, "dim", dim, T1[i])
    ks_subcort, values_subcort, regions_subcort = enrichment_function_subcortical(enrichment_subcort, get_ranks_subcortical(T1[i]))
    plot_subcortical(values_subcort, regions_subcort, ks_subcort, dim, "T1")
    ks_cort, values_cort, regions_cort = enrichment_function_cortical(enrichment_cort, get_ranks_cortical(T1[i]))
    plot_cortical(values_cort, regions_cort, ks_cort, dim, mod="T1")